In [1]:
import os
import cv2
import math
#import optuna
import h5py
from hyperopt import hp, fmin, tpe
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras import models
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Flatten, GlobalAveragePooling2D, Conv2D, Conv1D, MaxPooling2D, Reshape, BatchNormalization, RepeatVector, TimeDistributed, Add, Dropout, Attention
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, TimeDistributed, Concatenate, Embedding, concatenate
from tensorflow.keras.optimizers import Adam, Adadelta, Adamax, Nadam, Ftrl, SGD, RMSprop
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, Callback
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.metrics import CategoricalAccuracy, MeanSquaredError
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredError, categorical_crossentropy, mean_squared_error
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.resnet import ResNet50, ResNet101
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.nasnet import NASNetLarge
from tensorflow.keras.applications import EfficientNetB5
from typing import Optional
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras import regularizers
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.metrics import classification_report
from tensorflow.keras.metrics import Accuracy, Precision
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, precision_score, classification_report
#from focal_loss import categorical_focal_loss

In [2]:
final_h5_file_location = 'h5/'
final_h5_filename = 'data_6.h5'
final_h5_file_data_location = os.path.join(final_h5_file_location,
                                           final_h5_filename)

h5_file = h5py.File(final_h5_file_data_location, 'r')
x_train_data = h5_file.get('X_train')
x_train = x_train_data[()]
#print(x_train)
x_val_data = h5_file.get('X_val')
x_val = x_val_data[()]


y_train_data = h5_file.get('y_train')
y_train = y_train_data[()]
#print(x_train)
y_val_data = h5_file.get('y_val')
y_val = y_val_data[()]


In [3]:
print(type(x_train))
print(x_train.shape)
print(type(y_train))
print(y_train.shape)
h10= x_train.shape
h11 = x_train.shape[1]
print('h11',h11)

<class 'numpy.ndarray'>
(45345, 6, 20)
<class 'numpy.ndarray'>
(45345, 1)
h11 6


In [4]:
print(type(x_val))
print(x_val.shape)
print(type(y_val))
print(y_val.shape)

<class 'numpy.ndarray'>
(6478, 6, 20)
<class 'numpy.ndarray'>
(6478, 1)


In [5]:
class DualBlock(tf.keras.layers.Layer):
    def __init__(self, gamma_initializer='zeros',
                 gamma_regularizer=None,
                 gamma_constraint=None,
                 **kwargs):
        super(DualBlock, self).__init__(**kwargs)
        
        self.gamma_initializer = tf.keras.initializers.get(gamma_initializer)
        self.gamma_regularizer = tf.keras.regularizers.get(gamma_regularizer)
        self.gamma_constraint = tf.keras.constraints.get(gamma_constraint)

    def build(self, input_shape):
        _, h, filters = input_shape
        self.conv_b = Conv1D(filters // 8, 1, use_bias=False, kernel_initializer='he_normal')
        self.conv_c = Conv1D(filters // 8, 1, use_bias=False, kernel_initializer='he_normal')
        self.conv_d = Conv1D(filters, 1, use_bias=False, kernel_initializer='he_normal')
        self.gamma = self.add_weight(shape=(1,),
                                     initializer=self.gamma_initializer,
                                     name='gamma',
                                     regularizer=self.gamma_regularizer,
                                     constraint=self.gamma_constraint)
          
        super(DualBlock, self).build(input_shape)
        
    def pam_layer(self, input):
   
        input_shape = input.get_shape().as_list()
        _, h, filters = input_shape

        b = self.conv_b(input)
        c = self.conv_c(input)
        d = self.conv_d(input)

        vec_b = K.reshape(b, (-1, h, filters // 8))
        vec_cT = tf.transpose(K.reshape(c, (-1, h, filters // 8)), (0, 2, 1))
        bcT = K.batch_dot(vec_b, vec_cT)
        softmax_bcT = Activation('softmax')(bcT)
        vec_d = K.reshape(d, (-1, h, filters))
        bcTd = K.batch_dot(softmax_bcT, vec_d)
        bcTd = K.reshape(bcTd, (-1, h, filters))

        out = self.gamma*bcTd + input
        return out

    def cam_layer(self, input):
        input_shape = input.get_shape().as_list()
        _, h, filters = input_shape

        vec_a = K.reshape(input, (-1, h, filters))
        vec_aT = tf.transpose(vec_a, (0, 2, 1))
        aTa = K.batch_dot(vec_aT, vec_a)
        softmax_aTa = Activation('softmax')(aTa)
        aaTa = K.batch_dot(vec_a, softmax_aTa)
        aaTa = K.reshape(aaTa, (-1, h, filters))

        out = self.gamma*aaTa + input
        return out

    def compute_output_shape(self, input_shape):
        return input_shape

    def call(self, input):
        pam_feature = self.pam_layer(input)
        cam_feature = self.cam_layer(input)
        dual_feature = pam_feature + cam_feature
        return dual_feature
    
    def get_config(self):
        config = super(DualBlock, self).get_config()
        config.update({
            'gamma_initializer': tf.keras.initializers.serialize(self.gamma_initializer),
            'gamma_regularizer': tf.keras.regularizers.serialize(self.gamma_regularizer),
            'gamma_constraint': tf.keras.constraints.serialize(self.gamma_constraint)
        })
        return config

In [6]:
#Define model
def build_model(input_shape): 
    input_layer = Input(shape=input_shape) 
    h1 = h11
    h2 = 20
 
    lstm1 = Bidirectional(LSTM(units=128, input_shape=(h1, h2), return_sequences=True))(input_layer)
    print('lstm1', lstm1.shape)

    cbc0 = DualBlock()
    cb0 = cbc0(lstm1)


    #reshaped_inputp0 = Reshape((h1, h2))(x)
    lstm2 = Bidirectional(LSTM(units=256,input_shape=(h1, h2), return_sequences=True))(input_layer)

    cbp0 = DualBlock()
    cp0 = cbp0(lstm2)

    cc0 = concatenate([cb0,cp0])
    #cc0 = Add()([cb0,cp0])
    print('cc0', cc0.shape)

    h3 =h11
    h4 =256

    lstm3 = Bidirectional(LSTM(units=128, input_shape=(h3,  h4), return_sequences=True))(cc0)
    print('lstm3', lstm3.shape)

    cbc1 = DualBlock()
    cb1 = cbc1(lstm3)
    print('cb1', cb1.shape)


    lstm4 = Bidirectional(LSTM(units=256,input_shape=(h3,  h4), return_sequences=True))(cc0)

    cbp1 = DualBlock()
    cp1 = cbp1(lstm4)

    cc1 = concatenate([cb1,cp1])
    #cc1 = Add()([cb1,cp1])

    print('cc1', cc1.shape)
    
    cc2 = concatenate([cc0,cc1])
    #cc2 = concatenate([reshaped_inputc0,cc1])
    print('cc2', cc2.shape)

    lstm5 = Bidirectional(LSTM(units=128, input_shape=(h3,  h4), return_sequences=True))(cc2)
    cbc5 = DualBlock()
    cb5 = cbc5(lstm5)
    

    lstm6 = Bidirectional(LSTM(units=256, input_shape=(h3,  h4), return_sequences=True))(cc2)
    cbc6 = DualBlock()
    cb6 = cbc6(lstm6)
    print('lstm6', lstm6.shape)
    cc3 = concatenate([cb5,cb6])
    cc4 = concatenate([cc2,cc3])

    cc9 = Flatten()(cc4)


    # 定义边界框坐标预测输出层
    dense_output = Dense(512, activation='hard_sigmoid')(cc9)
    output = Dense(1)(dense_output)
    #output = Reshape(1, name='coordinate_output')(dense_output)

    model = Model(inputs=input_layer, outputs=output)
    return model

In [7]:
def train_model(model, X_train, Y_train, X_val, Y_val, batch_size, epochs):
   
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, min_lr=0.0000001)
    
   
    model.compile(optimizer=Nadam(learning_rate=0.00001), loss='mse', metrics=['mae', 'mse'])
   
    model.fit(x=X_train, y=Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, Y_val), callbacks=[reduce_lr])

In [8]:
# 初始化模型
#image_height, image_width, image_channels = image_data[0].shape
input_shape = (h11, 20)  # 根据图像大小进行设置
model = build_model(input_shape)#, num_actions, num_coordinates, num_frames,  num_targets)

# 打印模型摘要
model.summary()

lstm1 (None, 6, 256)
cc0 (None, 6, 768)
lstm3 (None, 6, 256)
cb1 (None, 6, 256)
cc1 (None, 6, 768)
cc2 (None, 6, 1536)
lstm6 (None, 6, 512)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 6, 20)]      0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 6, 256)       152576      input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 6, 512)       567296      input_1[0][0]                    
__________________________________________________________________________________________________
dual_block (DualBlock)          (None, 6, 256)       

In [9]:
batch_size = 8
epochs = 20
train_model(model, x_train, y_train,x_val, y_val, batch_size=batch_size, epochs=epochs)

Epoch 1/20
5669/5669 [==============================] - 363s 61ms/step - loss: 0.0012 - mae: 0.0211 - mse: 0.0012 - val_loss: 0.0024 - val_mae: 0.0430 - val_mse: 0.0024
Epoch 2/20
5669/5669 [==============================] - 343s 60ms/step - loss: 4.0250e-04 - mae: 0.0119 - mse: 4.0250e-04 - val_loss: 2.4403e-04 - val_mae: 0.0070 - val_mse: 2.4403e-04
Epoch 3/20
5669/5669 [==============================] - 344s 61ms/step - loss: 1.8660e-04 - mae: 0.0086 - mse: 1.8660e-04 - val_loss: 3.2199e-04 - val_mae: 0.0164 - val_mse: 3.2199e-04
Epoch 4/20
5669/5669 [==============================] - 342s 60ms/step - loss: 1.0210e-04 - mae: 0.0069 - mse: 1.0210e-04 - val_loss: 9.2929e-04 - val_mae: 0.0300 - val_mse: 9.2929e-04
Epoch 5/20
5669/5669 [==============================] - 345s 61ms/step - loss: 6.5280e-05 - mae: 0.0057 - mse: 6.5280e-05 - val_loss: 5.5330e-05 - val_mae: 0.0064 - val_mse: 5.5330e-05
Epoch 6/20
5669/5669 [==============================] - 341s 60ms/step - loss: 4.9124e-05 -

In [10]:
model.save('Nadam_6_DANet.h5', save_format='tf')